# 5일차 3교시 - Spark Cache, Persist and Unpersist
>  데이터 캐싱 즉 메모리에 저장하여 성능을 향상 시키는 방식에 대해 이해하고 적용합니다. 기본적으로  스파크는 필요한 데이터 소스의 경우 리니지를 통해 다시 계산하는 것을 원칙으로 하므로 자주 사용되는 중간 데이터의 경우 메모리에 저장해둔다면 유용할 수 있습니다. 일반 ETL 작업 보다는 그래프 연산이나 기계학습 등에 반복적인 처리과정에 효과적입니다.

### 목차
* [1. SparkSQL 을 통한 캐싱](#1.-SparkSQL-을-통한-캐싱)
* [2. Structured API 통한 캐싱](#2.-Structured-API-통한-캐싱)
* [3. Cache vs. Persist 비교](#3.-Cache-vs.-Persist-비교)
* [4. 캐시 전략](#4.-캐시-전략)

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

21/08/21 09:02:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/21 09:02:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. SparkSQL 통한 캐싱

```python
# RDD 자체를 캐싱하는 cache 혹은 persist 연산과는 다르게 즉시 테이블을 캐싱합니다.
spark.sql("CACHE TABLE [tableName]")
# 물론 LAZY 키워드를 이용하면 지연된 캐싱도 가능합니다
spark.sql("CACHE LAZY TABLE [tableName]")
# 캐시 테이블을 다시 리프래시 할 수 있습니다
spark.sql("REFRESH TABLE [tableName]")
# // 캐시로부터 해당 테이블을 제거합니다
spark.sql("UNCACHE TABLE IF EXISTS [tableName]")
# // 캐시로부터 모든 테이블들을 제거합니다
spark.sql("CLEAR CACHE")
```

In [2]:
from pyspark.sql.functions import *
t1 = spark.range(1, 1000)
t1.createOrReplaceTempView("t1")
spark.sql("cache table t1")

""


### 2. Structured API 통한 캐싱

#### 2.1. Cache 또한 Transformation 입니다.
> 즉, Lazy Evaluation 으로 동작하며, 그 즉시 캐싱되지 않습니다. (dataframe.cache() != cache table)
Segment of Logical Plan → CacheManager → Cached Data 순서대로 동작합니다.


* 스칼라를 통한 캐시 적용 및 플랜을 확인하는 방법 (파이썬에서는 동작하지 않습니다)
```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder.appName("scala-cache").getOrCreate()
val data = spark.range(1).cache()
println(data.queryExecution.withCachedData.numberedTreeString)
```
<br>

* 출력된 내용은 다음과 같습니다.
```bash
00 InMemoryRelation [id#0L], StorageLevel(disk, memory, deserialized, 1 replicas)
01    +- *(1) Range (0, 1, step=1, splits=8)
```
<br>

* 변환의 플랜을 확인하고자 한다면
```scala
data.withColumn("newId", 'id).explain(extended = true)
```
<br>

* 플랜 내역을 확인해봅니다.
```text
== Parsed Logical Plan ==
  'Project [id#32L, 'id AS newId#34]
    +- Range (0, 1, step=1, splits=Some(8))
== Analyzed Logical Plan ==
id: bigint, newId: bigint
  Project [id#32L, id#32L AS newId#34L]
    +- Range (0, 1, step=1, splits=Some(8))
== Optimized Logical Plan ==
  Project [id#32L, id#32L AS newId#34L]
    +- InMemoryRelation [id#32L], StorageLevel(disk, memory, deserialized, 1 replicas)
      +- *(1) Range (0, 1, step=1, splits=8)
== Physical Plan ==
  *(1) Project [id#32L, id#32L AS newId#34L]
    +- *(1) InMemoryTableScan [id#32L]
      +- InMemoryRelation [id#32L], StorageLevel(disk, memory, deserialized, 1 replicas)
        +- *(1) Range (0, 1, step=1, splits=8)
```
<br>

* 현재 캐시 여부를 확인하고자 한다면
```scala
val cache = spark.sharedState.cacheManager
cache.lookupCachedData(data.queryExecution.logical).isDefined
```

In [3]:
s1 = spark.read.parquet("source/s1")
s2 = spark.read.parquet("source/s2")

In [4]:
s1.printSchema()
s2.printSchema()
s1.show(1)
s2.show(1)

root
 |-- registration: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engine_size: decimal(38,18) (nullable = true)

root
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engine_size: decimal(38,18) (nullable = true)
 |-- sale_price: double (nullable = true)

+------------+----+------+--------------------+
|registration|make| model|         engine_size|
+------------+----+------+--------------------+
|     CPbYgbw|FORD|FIESTA|1.300000000000000000|
+------------+----+------+--------------------+
only showing top 1 row

+----+-----+--------------------+----------+
|make|model|         engine_size|sale_price|
+----+-----+--------------------+----------+
|FIAT|  500|1.100000000000000000|    1610.0|
+----+-----+--------------------+----------+
only showing top 1 row



### 3. Cache vs. Persist 비교

#### 3.1 Cache 
* RDD cache() 메소드는 기본이 MEMORY_ONLY 캐싱이며 
* DataFrame, DataSet cache() 메소드는 기본이 MEMORY_AND_DISK 캐싱입니다
* cache 메소드는 결국 내부적으로 persist 함수를 호출하며 sparkSession.sharedState.cacheManger.cacherQuery 에서 관리됩니다.

#### 3.2 Persist
* persist() 메소드는 STORAGE 레벨을 직접 지정할 수 있습니다
* DataFrame 혹은 DataSet 의 cache() 함수는 persist(StorageLevel.MEMORY_AND_DISK)와 동일합니다

#### 3.3 Unpersis
* 캐시된 데이터를 제거하며 모든 블록이 제거될 때 까지 블록되는 옵션을 가집니다 unpersist(default=true)
```python
cachedData = df.cache()
cachedData.show()
cachedData.unpersist()  # 당연하겠지만 df 또한 unpersist 됩니다
```

#### 3.4 데이터프레임을 통한 캐싱 여부 확인
* c12 StorageLevel(disk, memory, deserialized, 1 replicas)
* StorageLevel(memory, 1 replicas)
* 참고로 spark 은 replication 을 직접 구현하지 않으며 hdfs 의 것을 그대로 활용합니다

In [5]:
from pyspark import StorageLevel
c12 = s1.join(s2, [s1.make == s2.make, s1.model == s2.model])
unpersist = c12.where(col("sale_price") > 2000.0).unpersist()
unpersist.explain()  # Default StorageLevel(disk, memory, deserialized, 1 replicas)

== Physical Plan ==
*(2) BroadcastHashJoin [make#37, model#38], [make#44, model#45], Inner, BuildLeft, false
:- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false], input[2, string, false]),false), [id=#104]
:  +- *(1) Filter (isnotnull(make#37) AND isnotnull(model#38))
:     +- *(1) ColumnarToRow
:        +- FileScan parquet [registration#36,make#37,model#38,engine_size#39] Batched: true, DataFilters: [isnotnull(make#37), isnotnull(model#38)], Format: Parquet, Location: InMemoryFileIndex[file:/home/jovyan/work/lgde-spark-troubleshoot/source/s1], PartitionFilters: [], PushedFilters: [IsNotNull(make), IsNotNull(model)], ReadSchema: struct<registration:string,make:string,model:string,engine_size:decimal(38,18)>
+- *(2) Filter (((isnotnull(sale_price#47) AND (sale_price#47 > 2000.0)) AND isnotnull(make#44)) AND isnotnull(model#45))
   +- *(2) ColumnarToRow
      +- FileScan parquet [make#44,model#45,engine_size#46,sale_price#47] Batched: true, DataFilters: [isnotnul

In [6]:
cachedDiskMemory = c12.cache()
cachedDiskMemory.explain()  # StorageLevel(disk, memory, deserialized, 1 replicas)

== Physical Plan ==
InMemoryTableScan [registration#36, make#37, model#38, engine_size#39, make#44, model#45, engine_size#46, sale_price#47]
   +- InMemoryRelation [registration#36, make#37, model#38, engine_size#39, make#44, model#45, engine_size#46, sale_price#47], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(2) BroadcastHashJoin [make#37, model#38], [make#44, model#45], Inner, BuildLeft, false
            :- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false], input[2, string, false]),false), [id=#143]
            :  +- *(1) Filter (isnotnull(make#37) AND isnotnull(model#38))
            :     +- *(1) ColumnarToRow
            :        +- FileScan parquet [registration#36,make#37,model#38,engine_size#39] Batched: true, DataFilters: [isnotnull(make#37), isnotnull(model#38)], Format: Parquet, Location: InMemoryFileIndex[file:/home/jovyan/work/lgde-spark-troubleshoot/source/s1], PartitionFilters: [], PushedFilters: [IsNotNull(make), IsNotNul

In [7]:
cachedMemoryOnly = c12.where(col("sale_price") > 2100.0).persist(StorageLevel.MEMORY_ONLY)
cachedMemoryOnly.explain()  # StorageLevel(memory, 1 replicas)

== Physical Plan ==
InMemoryTableScan [registration#36, make#37, model#38, engine_size#39, make#44, model#45, engine_size#46, sale_price#47]
   +- InMemoryRelation [registration#36, make#37, model#38, engine_size#39, make#44, model#45, engine_size#46, sale_price#47], StorageLevel(memory, 1 replicas)
         +- *(1) Filter (isnotnull(sale_price#47) AND (sale_price#47 > 2100.0))
            +- InMemoryTableScan [registration#36, make#37, model#38, engine_size#39, make#44, model#45, engine_size#46, sale_price#47], [isnotnull(sale_price#47), (sale_price#47 > 2100.0)]
                  +- InMemoryRelation [registration#36, make#37, model#38, engine_size#39, make#44, model#45, engine_size#46, sale_price#47], StorageLevel(disk, memory, deserialized, 1 replicas)
                        +- *(2) BroadcastHashJoin [make#37, model#38], [make#44, model#45], Inner, BuildLeft, false
                           :- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false], input[2, str

### 4. 캐시 전략
* SparkSQL 은 테이블 단위 캐시와, Lazy 하지 않은 캐시인 점을 이용하면 좀 더 직관적인 관리 및 운영이 가능합니다
* Dataframe, Dataset 은 Lazy Evaluation 인 점을 감안하면, 보다 정교한 개발 및 모듈 개발이 가능합니다 (최적화 관점)
* 메모리에 올라와 있지만 캐시가 어떤 이유로든 삭제된 경우 transformation 연산을 통해 다시 계산됩니다
* 스파크는 모든 persist(), cache() 메소드가 호출될 때마다 전체 메모리를 모니터링합니다
* 해당 노드에 사용되지 않거나 혹은 LRU 알고리즘에 따라 캐시 데이터를 삭제합니다.

* Storage Level 수준에 따른 리소스 사용 매트릭스

```bash
Storage Level    Space used  CPU time  In memory  On-disk  Serialized   Recompute some partitions
----------------------------------------------------------------------------------------------------
MEMORY_ONLY          High        Low       Y          N        N         Y    
MEMORY_ONLY_SER      Low         High      Y          N        Y         Y
MEMORY_AND_DISK      High        Medium    Some       Some     Some      N
MEMORY_AND_DISK_SER  Low         High      Some       Some     Y         N
DISK_ONLY            Low         High      N          Y        Y         N
```

* 스토리지 수준에 따른 구현
  * MEMORY_ONLY         → RDD 를 Java Object 로 역직렬화 해서 저장하며 메모리가 부족한 경우 모든 단계를 다시 계산해서 사용합니다
  * MEMORY_ONLY_2       → MEMORY_ONLY 와 동일하지만 replica 수가 2입니다
  * MEMORY_ONLY_SER     → RDD 의 Java Object 형태로 메모리에 저장하지 않고 직렬화된 객체를 메모리에 저장하여 메모리 사용에 조금 더 효과적입니다
  * MEMORY_AND_DISK_SER → 메모리가 부족한 경우 부족한 데이터에 대해 디스크에 저장하되, 자바 객체 대신 직렬화된 데이터를 저장합니다
  * DISK_ONLY           → 디스크에만 저장합니다